Link: https://github.com/nbertagnolli/counsel-chat

### Load counsel-chat data

In [1]:
from datasets import load_dataset

/Users/abhi/miniconda3/envs/zoo/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = load_dataset("nbertagnolli/counsel-chat")['train'].to_pandas()

Repo card metadata block was not found. Setting CardData to empty.


In [3]:
data.head(2)

,questionID,questionTitle,questionText,questionLink,topic,therapistInfo,therapistURL,answerText,upvotes,views
0,0,Do I have too many issues for counseling?,I have so many issues to address. I have a his...,https://counselchat.com/questions/do-i-have-to...,depression,Jennifer MolinariHypnotherapist & Licensed Cou...,https://counselchat.com/therapists/jennifer-mo...,It is very common for people to have multiple ...,3,1971
1,0,Do I have too many issues for counseling?,I have so many issues to address. I have a his...,https://counselchat.com/questions/do-i-have-to...,depression,"Jason Lynch, MS, LMHC, LCAC, ADSIndividual & C...",https://counselchat.com/therapists/jason-lynch...,"I've never heard of someone having ""too many i...",2,386


In [4]:
data.shape

(2775, 10)

In [20]:
# we can potentially predict these titles out too given text – could help us get to the "RFV" – reason for visit perhaps?
data['questionTitle'].value_counts()

questionTitle
How would I know if I have the right therapist?                                   107
Do I have too many issues for counseling?                                          86
Is it normal to cry at therapy?                                                    83
I think my daughter is stressing too much                                          51
Can I change my feeling of being worthless to everyone?                            35
                                                                                 ... 
How do I get out of an abusive relationship?                                        1
I have a son with my girlfriend, but she is emotionally abusive. What do I do?      1
When should I give up on a relationship?                                            1
I endured sexual and physical abuse by my brother as child                          1
I need to know how to cope with misophonia before I go completely insane            1
Name: count, Length: 933, dtype: int64

In [5]:
data['topic'].value_counts()

topic
depression                  465
anxiety                     358
counseling-fundamentals     270
intimacy                    248
relationships               202
parenting                   191
family-conflict             144
trauma                      102
self-esteem                 100
relationship-dissolution     98
behavioral-change            69
lgbtq                        65
marriage                     52
anger-management             48
spirituality                 47
substance-abuse              45
professional-ethics          41
grief-and-loss               37
workplace-relationships      36
social-relationships         25
diagnosis                    25
domestic-violence            21
eating-disorders             16
legal-regulatory             14
stress                       13
addiction                    13
sleep-improvement            11
children-adolescents          8
human-sexuality               7
military-issues               3
self-harm                     1
Na

In [12]:
# observation: these topics could be classes itself for our model
# problem: multi-label so one example could span multiple classes but our examples might only have label each
# solution: we could train the model on this and then utilize Sigmoid to get multi-label predictions out during inference

In [14]:
# let's find out how many unique therapists we have here
# cool, we can utilize each therapists responses to build their profiles and learn their styles
data['therapistInfo'].value_counts()

therapistInfo
Sherry Katz, LCSWCouples and Family Therapist, LCSW                                                                       497
Lauren Ostrowski, MA, LPC, NCC, BC-TMH, CCTP, CCHI tailor my therapeutic approach to each client's strengths and goals    146
Lynda MartensMarriage & Family Therapist, MSc, RP, RMFT                                                                   127
Kaileen McMickle, MS, LPCLicensed Professional Counselor                                                                   84
Dr. Meredyth LawrynceServing Clients Nationwide                                                                            64
                                                                                                                         ... 
Sally highNo more coping skills. It's time to eliminate fear.                                                               1
Rachel LingleWholeHeartAZ.com Chandler                                                                  

In [ ]:
# let's check if multiple therapists have commented on the same posts
data

In [16]:
# doesn't seem super useful
data['upvotes'].describe()

count    2775.000000
mean        0.519640
std         1.042825
min         0.000000
25%         0.000000
50%         0.000000
75%         1.000000
max        12.000000
Name: upvotes, dtype: float64

In [17]:
# could we use this as a proxy to how useful the response was? this is better than upvotes as there, the distribution is pretty skewed
data['views'].describe()

count     2775.000000
mean       328.031712
std        796.306059
min          1.000000
25%         74.500000
50%        132.000000
75%        254.500000
max      16738.000000
Name: views, dtype: float64

In [18]:
# cool, so looking at this dataset, we can do the following:
# 1. topic classifier (let's keep it simple with a sentence-BERT model here.)
# 2. therapist persona modeling with their data
# 3. could we perhaps also use the topic classification to model our patients after? might be more consistent – let's simulate 5 patients and each of them span 4-5 topics